In [1]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...


remote: Enumerating objects: 4056, done.
remote: Counting objects: 100% (4056/4056), done.
remote: Compressing objects: 100% (3084/3084), done.
remote: Total 4056 (delta 1185), reused 1938 (delta 915), pack-reused 0
Receiving objects: 100% (4056/4056), 54.70 MiB | 8.05 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [2]:
%%bash
cd /home/toonies/Learn/TensorBot-Vision/notebooks/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /home/toonies/Learn/TensorBot-Vision/notebooks/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696860 sha256=291c6849b9054f9ea74f1e6bc1f479b6a5aa2c7c70b4977ab85b442649c63696
  Stored in directory: /tmp/pip-ephem-wheel-cache-at7uh3s1/wheels/ef/16/13/3147b8a6eccb81c6931fe949fbde297c68410d097163e6d294
Successfully built object-detection
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 

In [4]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [5]:
!python /home/toonies/Learn/TensorBot-Vision/notebooks/models/research/object_detection/builders/model_builder_tf2_test.py

2023-11-25 08:30:30.457252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-25 08:30:32.488879: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Running tests under Python 3.9.13: /home/toonies/anaconda3/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W1125 08:30:32.544338 139640747077760 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
/home/toonies/anaconda3/lib/python3.9/site-packages/object_detection/bui

In [6]:
%cd /home/toonies/Learn/TensorBot-Vision/notebooks
#Download Training set from git by cloning rep:
import os
import pathlib
# Clone the training set repository if it doesn't already exist
if "RetrainModelExample" in pathlib.Path.cwd().parts:
  while "RetrainModelExample" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('RetrainModelExample').exists():
  !git clone --depth 1 https://github.com/KostaMalsev/RetrainModelExample
  %cd /content/RetrainModelExample/TrainingSet/Picka 
  !unzip Pickachu.v1.tfrecord.zip -d /content/

#NOTE: Update these TFRecord names to your files containing training set!
#Also, Update relevant rows:in training config file "ssd_mobilenet_v2_320x320_coco17_tpu-8.config"
#label_map_path,input_path 

test_record_fname = '/content/valid/pickachu-toy.tfrecord'
train_record_fname = '/content/train/pickachu-toy.tfrecord'
label_map_pbtxt_fname = '/content/train/pickachu-toy_label_map.pbtxt'

/home/toonies/Learn/TensorBot-Vision/notebooks
Cloning into 'RetrainModelExample'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.
[Errno 2] No such file or directory: '/content/RetrainModelExample/TrainingSet/Picka'
/home/toonies/Learn/TensorBot-Vision/notebooks
unzip:  cannot find or open Pickachu.v1.tfrecord.zip, Pickachu.v1.tfrecord.zip.zip or Pickachu.v1.tfrecord.zip.ZIP.


In [7]:
test_record_fname = '/home/toonies/Learn/TensorBot-Vision/data/test_od/tfrecord_data/test.record'
train_record_fname = '/home/toonies/Learn/TensorBot-Vision/data/test_od/tfrecord_data/train.record'
label_map_pbtxt_fname = '/home/toonies/Learn/TensorBot-Vision/data/test_od/label_map.pbtxt'

In [8]:

#You can change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'ssd_mobilenet_v2_320x320_coco17': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 16
    }
}
chosen_model = 'ssd_mobilenet_v2_320x320_coco17'
num_steps = 1800 #40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your trainin#g